In [1]:
import sys
import importlib
import pickle
import random

# data manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# data analysis
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# pytorch specific
import torch
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

# project specific
sys.path.append('../')
from src import config
from src.utils import visualisation, benchmark, helpers, monitoring, experiment, search
from src.models import model


from matplotlib.animation import FuncAnimation
from IPython.display import HTML

pd.options.display.width = 1000
DEVICE = torch.device(config["DEVICE"])

In [48]:
# We first set the parametters that we do no want to change

data_params = { 
    "LS_threshold" : 0.0023,
    "MAD_threshold" : 1, 
    "MT_removal" : True, 
    "expression_threshold" : 0.1}
# or we can use a pre loaded dataset to save time
data_params = '../workfiles/light_BRCA_ds.pkl'

model_params = {
    "convolution": True,
    "transformer" : False
    }



# This one is absurdly clumsy but that's the only way i could do it.

variational_param_ = {
    "variational": [ # constraint : embedding_dim <= num_embeddings
        (("embedding_dim", 16), ("num_embeddings", 64)),
        (("embedding_dim", 16), ("num_embeddings", 128)),
        (("embedding_dim", 16), ("num_embeddings", 256)),
        (("embedding_dim", 16), ("num_embeddings", 512)),
        (("embedding_dim", 32), ("num_embeddings", 64)),
        (("embedding_dim", 32), ("num_embeddings", 128)),
        (("embedding_dim", 32), ("num_embeddings", 256)),
        (("embedding_dim", 32), ("num_embeddings", 512)),
        (("embedding_dim", 64), ("num_embeddings", 64)),
        (("embedding_dim", 64), ("num_embeddings", 128)),
        (("embedding_dim", 64), ("num_embeddings", 256)),
        (("embedding_dim", 64), ("num_embeddings", 512)),
        (("embedding_dim", 128), ("num_embeddings", 128)),
        (("embedding_dim", 128), ("num_embeddings", 256)),
        (("embedding_dim", 128), ("num_embeddings", 512))]}




dynamic_params = {
    "dropout"       : [0.1, 0.3, 0.5],
    "n_epoch"       : [100, 500, 1000, 3000],
    "latent_dim"    : [16, 32, 64, 128],
    "latent_dim"    : [16, 32, 64, 128],
    "num_layers"    : [1, 2,3,4],
    # ... (any other uncoupled parameters)
    # Coupled parameters are passed as tuples within the list
    "conv_params": [
        (("padding", 3), ("kernel_size", 7)),
        (("padding", 2), ("kernel_size", 5)),
        (("padding", 1), ("kernel_size", 3))
    ],
    # Note: The key "conv_params" is only a placeholder and won't appear in the final configs.*
    "variational" : [],
}




for commitment_cost in [0.2, 0.4, 0.6, 0.8, 1]:
    for (param1, param2) in variational_param_['variational']:
        dynamic_params["variational"].append((("variational", "VQ-VAE"),("commitment_cost", commitment_cost),param1, param2))

# we want to be as likely to pick a VQ-VAE, VAE or non variational model.
# Here the tuple  ('commitment_cost', None) is used as a place holder so the algorithm doesn't freak out
dynamic_params['variational'] = dynamic_params['variational'] + [(("variational", None), ('commitment_cost', None))]*len(dynamic_params['variational']) + [(("variational", "VAE"), ('commitment_cost', None))]*len(dynamic_params['variational'])

configurations = helpers.generate_config(model_params, dynamic_params)
print("number of config to be evaluated:",len(configurations))

all_config = helpers.generate_config(model_params, dynamic_params)
print(len(all_config))

number of config to be evaluated: 129600
129600


In [49]:
dynamic_params['variational']

[(('variational', 'VQ-VAE'),
  ('commitment_cost', 0.2),
  ('embedding_dim', 16),
  ('num_embeddings', 64)),
 (('variational', 'VQ-VAE'),
  ('commitment_cost', 0.2),
  ('embedding_dim', 16),
  ('num_embeddings', 128)),
 (('variational', 'VQ-VAE'),
  ('commitment_cost', 0.2),
  ('embedding_dim', 16),
  ('num_embeddings', 256)),
 (('variational', 'VQ-VAE'),
  ('commitment_cost', 0.2),
  ('embedding_dim', 16),
  ('num_embeddings', 512)),
 (('variational', 'VQ-VAE'),
  ('commitment_cost', 0.2),
  ('embedding_dim', 32),
  ('num_embeddings', 64)),
 (('variational', 'VQ-VAE'),
  ('commitment_cost', 0.2),
  ('embedding_dim', 32),
  ('num_embeddings', 128)),
 (('variational', 'VQ-VAE'),
  ('commitment_cost', 0.2),
  ('embedding_dim', 32),
  ('num_embeddings', 256)),
 (('variational', 'VQ-VAE'),
  ('commitment_cost', 0.2),
  ('embedding_dim', 32),
  ('num_embeddings', 512)),
 (('variational', 'VQ-VAE'),
  ('commitment_cost', 0.2),
  ('embedding_dim', 64),
  ('num_embeddings', 64)),
 (('variation

In [50]:
# Initialize GA parameters
population_size = 5

search_param = {
    "population_size" : population_size,    # this is very low for a genetic algo
    "mutation_rate" : 0.5,                  # Every run is so costly that we have to have an aggressively high value
    "crossover_rate" : 0.7,                 # usually high
    "generations" : 5                       # way too low for GA as well
    }


# Step 1: Create the initial population
population = random.sample(all_config, population_size)



In [51]:
individual = population[0]
individual 

{'convolution': True,
 'transformer': False,
 'dropout': 0.3,
 'n_epoch': 3000,
 'latent_dim': 16,
 'num_layers': 3,
 'padding': 1,
 'kernel_size': 3,
 'variational': 'VAE',
 'commitment_cost': None}

In [53]:
importlib.reload(search)

for i in [0]*1000:
    s = search.genetic_search(
        search_param=search_param, 
        data_param=data_params, 
        model_param=model_params, 
        initial_population= population,
        dynamic_params=dynamic_params)

    s.mutate(individual)

self.dynamic_params[mutation_param_key][0] (('variational', 'VQ-VAE'), ('commitment_cost', 0.2), ('embedding_dim', 16), ('num_embeddings', 64))
available_choices [(('variational', 'VQ-VAE'), ('commitment_cost', 0.2), ('embedding_dim', 16), ('num_embeddings', 64)), (('variational', 'VQ-VAE'), ('commitment_cost', 0.2), ('embedding_dim', 16), ('num_embeddings', 128)), (('variational', 'VQ-VAE'), ('commitment_cost', 0.2), ('embedding_dim', 16), ('num_embeddings', 256)), (('variational', 'VQ-VAE'), ('commitment_cost', 0.2), ('embedding_dim', 16), ('num_embeddings', 512)), (('variational', 'VQ-VAE'), ('commitment_cost', 0.2), ('embedding_dim', 32), ('num_embeddings', 64)), (('variational', 'VQ-VAE'), ('commitment_cost', 0.2), ('embedding_dim', 32), ('num_embeddings', 128)), (('variational', 'VQ-VAE'), ('commitment_cost', 0.2), ('embedding_dim', 32), ('num_embeddings', 256)), (('variational', 'VQ-VAE'), ('commitment_cost', 0.2), ('embedding_dim', 32), ('num_embeddings', 512)), (('variational'

In [26]:
importlib.reload(search)
s = search.genetic_search(
    search_param=search_param, 
    data_param=data_params, 
    model_param=model_params, 
    initial_population= population,
    dynamic_params=dynamic_params)

s.mutate(individual)

{'convolution': True,
 'transformer': False,
 'dropout': 0.3,
 'n_epoch': 3000,
 'latent_dim': 128,
 'num_layers': 3,
 'padding': 3,
 'kernel_size': 7,
 'variational': None}

{'convolution': True,
 'transformer': False,
 'dropout': 0.3,
 'n_epoch': 3000,
 'latent_dim': 64,
 'num_layers': 3,
 'padding': 3,
 'kernel_size': 7,
 'variational': 'VQ-VAE',
 'commitment_cost': 1,
 'embedding_dim': 32,
 'num_embeddings': 256}